Notes:
    
    2.) Type? Add in ID or STUDY in output    
    3.) Time input should be changed to hrs for consistency 

# Neural PK Walkthrough

This code is a rewrite of the code available at https://github.com/jameslu01/Neural_PK

We have streamlined the code, and arranged its main path in this notebook.

Note that the original paper and code have two different settings:
1. Random splits of training and test.
2. Cross-dosing splits. 

This notebook covers both settings. Whereas the original code base treats the two cases completely separately, this refactoring consolidates most of the functionality between the two, with minor differences as described below. 

In [1]:
import os
import torch
import pandas as pd
import data_parse_Ping
from data_split_Ping import data_split, data_split_cross, augment_data
from model_utils_Ping import train_neural_ode, predict_using_trained_model, merge_predictions
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from scipy.stats import pearsonr
import random
import torch.optim as optim

In [ ]:
# if you have a GPU and CUDA and pytorch with GPU support installed, this will come back as true
# it's ok if you don't have CUDA - you can still run all of the model code, but it will be slower
torch.cuda.is_available()

# Set Global Variables

In [2]:
"""
In this cell, we set and describe the various parameters and hyperparameters that are used throghout the code.

Note you'll find a few more parameters of the neural network hardcoded in model.py.
"""

BASE_RANDOM_SEED = 1329  # random seed for data splitting
TORCH_RANDOM_SEED = 1000  # they have different random seeds for splitting and for the neural network
SPLIT_FRAC = 0.2  # fraction of data reserved for testing
OUTER_FOLDS = [1, 2, 3, 4, 5]  # indices of train/test splits
CROSS_SCHED_FOLD = 999 # single fold value for the cross-schedule tran/test split used for setting seed and naming files
MODEL_REPLICATES = [1, 2, 3, 4, 5 ]  # indices of model replicates for the ensemble of neural ODEs

# hyperparemeters for the model, selected by grid search
# note: the paper was not clear WHICH hyperparameters were selected by grid search
LR = 0.001  # this is the most important hyperparameter to tune
L2 = 0.0001  # weight decay is a form of regularization. should be tuned
"""
ODE solvers can approximately ensure that the output is within a given tolerance of the true solution. 
The time spent by the forward call is proportional to the number of function evaluations, 
so tuning the tolerance gives us a trade-off between accuracy and computational cost. 
Our framework allows the user to trade off speed for precision, 
but requires the user to choose an error tolerance on both the forward and reverse passes during training. 
For sequence modeling, the default value of 1.5e-8 was used. In the classification and density estimation experiments, 
we were able to reduce the tolerance to 1e-3 and 1e-5, respectively, without degrading performance.
In short, tol is the tolerance for accepting/rejecting an adaptive step.
"""
TOL = 0.001
# number of epochs to train the model. the authors use early stopping so it's not crucial.
# just needs to be large enough so the validation loss decreases and then eventually increases
# allowing us to identify the model state with the local minimum
EPOCHS = 30
# all of these together decide the size of the neural network
HIDDEN_DIM = 32
LATENT_DIM = 16
ODE_HIDDEN_DIM = 16

# Loading and Processing Data

In [3]:
"""
Example data accompanying the paper. We'll import and preprocess in this cell.

Data has the following columns:
  - STUD - Study ID. Can be 1000, 2000, 3000.
  - PTNM - Patient number. Can be repeated between studies, but for example patient 1 in Study 1000 is not the same person as patient 1 in study 2000.
  - DSFQ - Dosage frequency is how often the dose is administred. Only 1 or 3.
  - AMT - Dosage amount. Can be 0 when measurements taken between doses.
  - TIME - Time since beginning of patient's treatment.
  - TFDS - Time since dose.
  - DV - Concentration measurement. 
"""
data_complete = pd.read_csv("ExampleData/ADCdosePK.csv", na_values=".")
select_cols = ["PTNM", "STUD", "TIME", "TFDS", "AMT", "DV"]
# According to authors: Patient data that have been marked with non-missing values in the "C" columns have been removed from the analysis
#if "C" in data_complete.columns.values:
    #data_complete = data_complete[data_complete.C.isnull()]
#data_complete = data_complete[data_complete.CYCL < 100]  # cut off all dosing cycles greater than 100
data_complete = data_complete[select_cols]  # filter down to columns of interest
data_complete = data_complete.rename(
    columns={"DV": "PK_timeCourse"}
) 

# DV is our variable of interest - anolyte concentration

data_complete["PTNM"] = data_complete["PTNM"].astype("int").map("{:05d}".format)
data_complete["ID"] = (
    data_complete["STUD"].astype("int").astype("str") + data_complete["PTNM"])  
print("The ID is returning",data_complete["ID"])
    #concatenate study ID and patient ID for overall, unique ID
   
    
time_summary = (
    data_complete[["ID", "TIME"]].groupby("ID").max().reset_index()
)  # get max time since start of treatment per ID

# only keep patients who have measurements past initial measurements (TIME == 0)
selected_ptnms = time_summary[time_summary.TIME > 0].ID

data_complete = data_complete[data_complete.ID.isin(selected_ptnms)]

data_complete["AMT"] = data_complete["AMT"].fillna(0)  # replace missing values for dosage with 0s

# Set up round 1 measurement features.
# Round 1 measurements for each ID are always used as input features for the neural network to predict measurements after round 1.
# For weekly dosage IDs, round 1 is anything before end of week 1 (TIME <= 168), for every 3 week dosage IDs, anything before end of week 3 (TIME <= 604)
#data_complete["PK_round1"] = data_complete["PK_timeCourse"]
#data_complete.loc[(data_complete.DSFQ == 1) & (data_complete.TIME >= 168), "PK_round1"] = 0
#data_complete.loc[(data_complete.DSFQ == 3) & (data_complete.TIME >= 504), "PK_round1"] = 0

# Missing PK measurement value handling
#data_complete["PK_round1"] = data_complete["PK_round1"].fillna(0)  # round 1 missing values filled with 0
data_complete["PK_timeCourse"] = data_complete["PK_timeCourse"].fillna(-1)  
#data=data_complete
#all others filled with -1, used to find missing values during training

#data_complete = data_complete[
#    ((data_complete.AMT == 0) & (data_complete.TIME == 0))
#]  # drop all first patient rows with no dosage

#data=data_complete
# Some rows are duplicate pairs for PTNM and TIME combinations with different cycle (CYCL) values
# Set the first dosage amount of duplicated rows to the last dosage amount and keep only last row of duplicated rows
# This implementation may be an issue if patient number (PTNM) repeats across multiple studies (STUD)
data_complete.loc[
    data_complete[["PTNM", "TIME"]].duplicated(keep="last"), "AMT"  # all non-last duplicated rows
] = data_complete.loc[
    data_complete[["PTNM", "TIME"]].duplicated(keep="first"), "AMT"  # all non-first duplicated rows
].values
data = data_complete[~data_complete[["PTNM", "TIME"]].duplicated(keep="first")]

The ID is returning 0       100001
1       100001
2       100001
3       100001
4       100001
         ...  
1481    400021
1482    400021
1483    400021
1484    400021
1485    400021
Name: ID, Length: 1486, dtype: object


# Implementation of Grid Search Hyperparameter Optimization

In [ ]:
import data_parse_Ping
import data_split_Ping

import data_parse_Ping
eval_results_all = {}
for fold in OUTER_FOLDS:
    for model in MODEL_REPLICATES:
        
        # first we split up the data into training/validation/test
        train, test = data_split(data, "PTNM", seed=BASE_RANDOM_SEED + fold, test_size=SPLIT_FRAC)
        train, validate = data_split(train, "PTNM", seed=BASE_RANDOM_SEED + fold + model, test_size=SPLIT_FRAC)
        
            
    test_add_to_train = pd.concat(
        [test[(test.PTNM.astype("int") >= 6)]], ignore_index=True
    )
    train = pd.concat([train, test_add_to_train], ignore_index=True)
    # i am not sure it makes sense to add this to the validation data?
    validate = pd.concat([validate, test_add_to_train], ignore_index=True)
        
    train = augment_data(train)
        
from model_Ping import Encoder, ODEFunc, Classifier
from model_utils_Ping import train_neural_ode


# define the range of hyperparameters to search over
param_grid = {
    "lr": [0.0001, 0.001, 0.01],
    "tol": [1e-3, 1e-4, 1e-5],
    "epochs": [30],
    "l2": [1e-4, 1e-3, 1e-2],
    "hidden_dim": [32, 64, 128],
    "latent_dim": [16, 32, 64],
    "ode_hidden_dim": [16, 32, 64]
}

# define the number of random search iterations
n_iter = 15

# define a function that will generate a random set of hyperparameters
def random_params(param_grid):
    params = {}
    for key in param_grid:
        params[key] = random.choice(param_grid[key])
    return params

# run the random search
for fold in OUTER_FOLDS:
    for i in range(n_iter):
        # generate a random set of hyperparameters
        params = random_params(param_grid)
    
        # train the model using the generated hyperparameters
        train_neural_ode(random_seed=42, train=train, validate=test, model=1, fold=6, lr = params['lr'], tol= params['tol'], epochs=30, l2=params['l2'],hidden_dim=params['hidden_dim'], latent_dim=params['latent_dim'], ode_hidden_dim=params['latent_dim'])
        eval_results = predict_using_trained_model(
            test,
            model,
            fold,
            TOL,
            HIDDEN_DIM,
            LATENT_DIM,
            ODE_HIDDEN_DIM,
        )
        print(f"Iteration {i+1}/{n_iter}: {params}")

# Random (Non-Cross Schedule) Split W/ 5-Fold CV

In [ ]:
"""
Here the data is split, the model is trained, and then evaluated on test data. The raw results
from this evaluatoin are saved.

This entire procedure is done multiple times due to multiple training/test splits and model averaging. 
The authors of this code are doing two things when it comes to splitting the data:

(1) They are doing 5 outer train/test splits. They repeat the entire training and test procedure 5 times.
This is good practice to establish variance of the test set performance.
(2) Within each of the 5 splits above, the are also doing model *averaging* by training 5 Neural ODE models
that differ in: (a) initial conditions, (b) random seeds and (c) which subset of the training data is used
for actual model training vs validation. These 5 models are then averaged together to get the final model
which is then applied to the test set. This is standard practice to improve model quality. It also brings up
an interesting methodological question: did they also use 5 model replicates for the other algorithms
that they benchmarked against? If not, then their model had an unfair advantage.

Feel free to skip this cell: we've already run the code and stored the results in CSVs.
"""
import data_parse_Ping
eval_results_all = {}
for fold in OUTER_FOLDS:
    for model in MODEL_REPLICATES:
            
         # first we split up the data into training/validation/test
        train, test = data_split(data, "PTNM", seed=BASE_RANDOM_SEED + fold, test_size=SPLIT_FRAC)
        train, validate = data_split(train, "PTNM", seed=BASE_RANDOM_SEED + fold + model, test_size=SPLIT_FRAC)
            
        """
        We don't know why this is happening. We asked the authors but they did not respond.             
        They are adding the first week of the test data to the training and validation data.
        Although this data is not used for evaluation (see the last few lines of merge_predictions in model_utils.py), 
        it is used for training the model. Our best guess is that this provides a bit of extra training data,
        but it's unrealistic as we won't have the first part of "real" test data in practice.
        """
        
        test_add_to_train = pd.concat(
            [test[(test.PTNM.astype("int") >= 15)]], ignore_index=True
        )
        train = pd.concat([train, test_add_to_train], ignore_index=True)
        # i am not sure it makes sense to add this to the validation data?
        validate = pd.concat([validate, test_add_to_train], ignore_index=True)
        
        """
        They add extra augmented data to the training set constructed out of existing training data. 
        Here is a description from the paper:
    
        "We applied augmentation to prevent overfitting.
        We applied timewise truncation to increase the number of training examples.
        For each training example, in addition to the original example, we also truncated
        the examples at 1008 hr, 1512 hr, and 2016 hr and generated and added
        a set of new examples to the training examples."
        """
            
        train = augment_data(train)
        train.fillna(0)
        # create and train the model
        # the best checkpoint will be saved
        train_neural_ode(
            TORCH_RANDOM_SEED + model + fold,
            train,
            validate,
            model,
            fold,
            LR,
            TOL,
            EPOCHS,
            L2,
            HIDDEN_DIM,
            LATENT_DIM,
            ODE_HIDDEN_DIM
        )

        # predict on test using the best model saved
        # during train_neural_ode
        eval_results = predict_using_trained_model(
            test,
            model,
            fold,
            TOL,
            HIDDEN_DIM,
            LATENT_DIM,
            ODE_HIDDEN_DIM,
        )

        eval_results_all[(fold, model)] = eval_results

In [ ]:
#No need to do this because we have eval_results_all from the previous cell, 
#but just a quick demonstration of how to load the predictions from the saved models
eval_results_all_loaded = {}

for fold in OUTER_FOLDS:
    for model in MODEL_REPLICATES:
        eval_path = os.path.join(f"fold_{fold}", f"fold_{fold}_model_{model}.csv")
        eval_results_all_loaded[(fold, model)] = pd.read_csv(eval_path)

In [ ]:
"""
Now we can compute evaluation metrics and summarize them

The paper discusses two of the metrics:

"We used R2 score, and Pearson correlation in this study. 
Correlation gives an intuitive estimation of the concordance between the predictions and ground truth,
while R2 also takes into account whether the overall scale of the predictions is correct."

However, the code also uses an additional metrics of root mean squared error (RMSE), which is a 
standard regression metric. It provides an additional view of the actual distance between the predictions
which is not available in R2 or correlation.

We have also added a new metric of the absolute difference between the predictions and the ground truth
because we believe it is a useful viewpoint that does not have the "squared" distortion of the RMSE.
"""
r2_scores = []
rmses = []
pearsonrs = []
abs_errors = []
for fold in OUTER_FOLDS:
    # perform the ensembling
    evals_per_fold = [eval_results_all_loaded[(fold, m)] for m in MODEL_REPLICATES]
    predictions = merge_predictions(evals_per_fold, data)
    # evaluate various metrics
    y_true = predictions["labels"].values
    y_pred = predictions["pred_agg"].values
    rmses.append(mean_squared_error(y_true, y_pred, squared=False))
    r2_scores.append(r2_score(y_true, y_pred))
    pearsonrs.append(pearsonr(y_true, y_pred)[0])
    abs_errors.append(mean_absolute_error(y_true, y_pred))


df = pd.DataFrame({"R2": r2_scores, "Pearson R": pearsonrs, "RMSE": rmses, "Mean Absolute Error": abs_errors})
df.index = OUTER_FOLDS
print(df, '\n')

summary_df = df.agg(["min", "max", "mean", "median"])
print(summary_df)

# Cross-Schedule Split

In [4]:
"""
Using the same building blocks, we can also train a model to predict cross-schedule PK response.

In this case, we only run one outer fold, but still train 5 models within this fold. Each of the 
models has the same test split (train and validate on 3-week dosing schedule, test on 1-week dosing 
schedule), but different train/validation splits (split randomly).

In this code, we have implemented the training and prediction of the split described above. The
original code creates 2 additional test sets:
(1) test_ineterp - the same test set, but with interpolated time points at 6 hour intervals. This was
    done to be able to generate predictions at a higher resolution.
(2) test_nodosing - the same test set with all dosing information masked (replaced with 0). This was
    used to produce the analysis described in Figure 4B.
"""
eval_results_all = {}
fold = CROSS_SCHED_FOLD # used for setting random seed and naming files
for model in MODEL_REPLICATES:

    # first we split up the data into training/validation/test
    # in this case, we split 1-week schedule patients to test,
    # then split the into train and validate randomly as above
    train, test = data_split_cross(data)
    train, validate = data_split(train, "PTNM", seed=BASE_RANDOM_SEED + fold + model, test_size=SPLIT_FRAC)

    # note that we do not add any of the test data to the train
    # and validate data as we did above

    # the training data is augmented identically as above
    train = augment_data(train)

    # create and train the model
    # the best checkpoint will be saved
    train_neural_ode(
        TORCH_RANDOM_SEED + model + fold,
        train,
        validate,
        model,
        fold,
        LR,
        TOL,
        EPOCHS,
        L2,
        HIDDEN_DIM,
        LATENT_DIM,
        ODE_HIDDEN_DIM,
    )

    # predict on test using the best model saved
    # during train_neural_ode
    eval_results = predict_using_trained_model(
        test,
        model,
        fold,
        TOL,
        HIDDEN_DIM,
        LATENT_DIM,
        ODE_HIDDEN_DIM,
    )

    eval_results_all[(fold, model)] = eval_results

C:\Users\AnneN\anaconda3\envs\neuralpk\lib\site-packages\pandas\core\indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
--fold 999 --model 1 --lr 0.001 --tol 0.001 --epochs 30 --l2 0.0001 --hidden_dim 32 --laten_dim 16
100%|##########| 176/176 [00:35<00:00,  4.90it/s]

            Epoch 0001 | Training loss 211.755112 | Training R2 0.304944 | Validation loss 1341.792114 | Validation R2 0.300108
            Best loss 1341.792114 | Best epoch 0001
            
100%|##########| 176/176 [00:42<00:00,  4.17it/s]

            Epoch 0002 | Training loss 995.364624 | Training R2 -14.357357 | Validation loss 1325.067993 | Validation R2 0.317446
            Best loss 1325.067993 | Best epoch 000

KeyboardInterrupt: 

In [ ]:
# No need to do this because we have eval_results_all from the previous cell, 
# but just a quick demonstration of how to load the predictions from the saved models
eval_results_all_loaded = {}

for model in MODEL_REPLICATES:
    eval_path = os.path.join(f"fold_{fold}", f"fold_{fold}_model_{model}.csv")
    eval_results_all_loaded[(fold, model)] = pd.read_csv(eval_path)

In [ ]:
"""
Now we can compute evaluation metrics same as before.
"""
# perform the ensembling
evals = [eval_results_all[(fold, m)] for m in MODEL_REPLICATES]
predictions = merge_predictions(evals, data)
# evaluate various metrics
y_true = predictions["labels"].values
y_pred = predictions["pred_agg"].values

rmse_val = mean_squared_error(y_true, y_pred, squared=False)
r2_score_val = r2_score(y_true, y_pred)
pearsonr_val = pearsonr(y_true, y_pred)[0]
abs_error_val = mean_absolute_error(y_true, y_pred)

metrics_vals = pd.Series({"R2": r2_score_val, "Pearson R": pearsonr_val, "RMSE": rmse_val, "Mean Absolute Error": abs_error_val})
print(metrics_vals)